<h2 style = "text-align:center" ><b>EEG</b> - Prédiction des Mouvements Imaginaires de la Main</h2>

---

#### **1. Le projet**
- Intoduction  
https://github.com/DataScientest-Studio/mar24_cds_eeg/blob/eric/references/Description_projet_EEG.pdf  
https://www.bbci.de/competition/iv/desc_2b.pdf
- Ressources / Données   
https://www.kaggle.com/competitions/ucsd-neural-data-challenge/overview  
- Bibliographie  
https://www.bbci.de/competition/iv/desc_2b.pdf
#### **2. Liens utils**
- SciPy - *open-source software for mathematics, science, and engineering*  
https://docs.scipy.org/doc/scipy/index.html  
https://docs.scipy.org/doc/scipy/reference/signal.html  
- MNE - *MEG + EEG Analysis & Visualisation*
   - Accueil  
   https://mne.tools/stable/index.html

   - MNE - Data structures from arbitrary data  
   https://mne.tools/stable/auto_tutorials/io/10_reading_meg_data.html#creating-mne-data-structures-from-arbitrary-data-from-memory
   
   - MNE - EEG Preprocessing  
   https://mne.tools/dev/auto_tutorials/preprocessing/index.html  

- pyRiemann - *Biosignals classification with Riemannian geometry*  
https://pyriemann.readthedocs.io/en/latest/  
- neurodsp - *Neuro Digital Signal Processing Toolbox*  
https://neurodsp-tools.github.io/neurodsp/index.html#
- Rythme Mu  
https://fr.wikipedia.org/wiki/Rythme_Mu
- Spectrogram from EEG  
https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg
- Divers  
https://signalprocessingsociety.org/  
https://fr.wikipedia.org/wiki/Filtre_de_Butterworth  
https://fr.wikipedia.org/wiki/Moyenne_mobile  
https://terpconnect.umd.edu/~toh/spectrum/Differentiation.html  
https://perso.etis-lab.fr/ghaffari/2014_CCMB_Floride_USA.pdf  
https://www.youtube.com/watch?v=wB417SAbdak&list=PLXc9qfVbMMN2TAoLHVW5NvNmJtwiHurzw  
https://fastercapital.com/fr/sujet/identification-des-artefacts-de-traitement-du-signal-dans-des-sc%C3%A9narios-r%C3%A9els.html#:~:text=L'inspection%20visuelle%20est%20la,des%20pertes%20et%20du%20bruit.  
   - Z-Score Normalisation  
   https://fr.wikipedia.org/wiki/Cote_Z_(statistiques)  
   https://typeset.io/questions/why-is-z-score-normalisation-necessary-in-pre-processing-eeg-1xv5jepyq5  

   - Traitement numérique du signal  
   https://fr.wikipedia.org/wiki/Traitement_num%C3%A9rique_du_signal  
   - Ondelette  
      - Wiki  
      https://fr.wikipedia.org/wiki/Ondelette  

      - L’analyse par ondelettes dans la vie de tous les jours  
      https://interstices.info/lanalyse-par-ondelettes-dans-la-vie-de-tous-les-jours/  

      - A guide for using the Wavelet Transform in Machine Learning  
      https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/
      
      - pyWavelets - *open source wavelet transform*  
      https://pywavelets.readthedocs.io/en/latest/

      - Ondelettes et applications  
      https://www.i2m.univ-amu.fr/~caroline.chaux/GEOMDATA/TI-te5215.pdf

   - Maximum de vraisemblance  
   https://pmarchand1.github.io/ECL8202/notes_cours/03-Maximum_vraisemblance.html  
   https://fr.wikipedia.org/wiki/Maximum_de_vraisemblance#:~:text=En%20statistique%2C%20l'estimateur%20du,maximisant%20la%20fonction%20de%20vraisemblance  

   - Transformation de Fourier discrète  
   https://fr.wikipedia.org/wiki/Transformation_de_Fourier_discr%C3%A8te  
      - La Transformation de Fourier n’est pas adaptée à l’analyse des signaux non stationnaires.
   - Neural Data Science in Python  
   https://neuraldatascience.io/intro.html

   - Preprocessing of EEG  
   https://www.frontiersin.org/articles/10.3389/fninf.2015.00016/full#:~:text=The%20depositable%20preprocessing%20pipeline%20consists,with%20a%20low%20recording%20SNR  
   https://typeset.io/papers/preprocessing-of-eeg-4go8vhcbty  
   https://learn.neurotechedu.com/preprocessing  
   https://g0rella.github.io/gorella_mwn/preprocessing_eeg.html  
   
   - Biblio :  
   https://perso.telecom-paristech.fr/bloch/P6Image/ondelettestrsp.pdf  
   https://www.math.u-bordeaux.fr/~jbigot/Site/Enseignement_files/ondelettesIMAT.pdf  
   http://w3.cran.univ-lorraine.fr/perso/radu.ranta/pdf/cours_deb_ond%28fr%29.pdf
   
   - Digital Filtering  
   http://notebooks.pluxbiosignals.com/notebooks/Categories/Pre-Process/digital_filtering_eeg_rev.html

   - Processus stationnaire  
   https://fr.wikipedia.org/wiki/Processus_stationnaire

   - Analyse en composantes principales  
   https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales#:~:text=L'ACP%2C%20d%C3%A9sign%C3%A9e%20en%20g%C3%A9n%C3%A9ral,une%20grandeur%20physique%2C%20comme%20les

#### • Chargement des différentes librairies

In [1]:
### • Chargement des différentes librairies

import sys, os, time, math

sys.path.insert(0, os.path.dirname(os.getcwd()))

from src.thot.mathesis import *

from sklearn import model_selection, preprocessing as sk_p
from sklearn import ensemble, svm, neighbors
# from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import classification_report, accuracy_score

from keras.models import Sequential                             # type: ignore
from keras.callbacks import EarlyStopping                       # type: ignore
from keras.layers import GlobalAveragePooling1D                 # type: ignore
from keras.layers import Dense, Dropout, Conv1D, LSTM           # type: ignore
from keras.layers import LeakyReLU, ReLU, PReLU, ConvLSTM1D     # type: ignore
# from keras.layers import Bidirectional, TimeDistributed, RepeatVector, Flatten

# import tensorflow as tf
# from tensorflow.keras.optimizers import AdamW, Adam            # type: ignore

# from pyriemann.spatialfilters import CSP

import pywt, librosa
import seaborn as sns

# from scipy.fft import fft

%matplotlib inline

#### • Déclaration de constantes

In [2]:
### • Déclaration de constantes

# Fréquence d'échantillonnage - Hz (Nombre de valeur / sec)
SAMPLE_RATE  = 250
# Temps additionel pour étendre le domaines d'étude.
LAG : int    = -62     # Décalage du signal dû signal ~250ms
#
PW2 : int    = int(np.floor(np.log2(SAMPLE_RATE))) # 2 << SAMPLE_RATE // 32
#
NFFT : int   = 1 << PW2
# Epoque en sec donnée en nombre d'échantillon consectutif # 4" de données (multiple de 2)
SCOPE : int  = SAMPLE_RATE << 1 # (1 << PW2) * 4
# Deux enregistrements bipolaires + neutre
eeg_Chans    = ['C3', 'C4', 'Cz']
# Liste des cannaux eeg associés aux évènement 0 et 1
eeg_left     = [f'{c}_0' for c in eeg_Chans]
eeg_right    = [f'{c}_1' for c in eeg_Chans]
full_eeg     = eeg_left + eeg_right
# Trois enregistrements musculaires
ecg_Chans    = ['EOG:ch01', 'EOG:ch02', 'EOG:ch03']
# Liste de tous les cannaux des dataframes
all_chans    = eeg_Chans + ecg_Chans
# Correspondance pour la classification
hands_event  = {0: 'Left', 1: 'Right'}
# Les bandes de fréquences d'intérêt
eeg_bands    = {'Delta' : (0.1, 4.0),
                'Theta' : (4.1, 8.0),
                'Alpha' : (8.1, 14.0),
                'Beta'  : (14.1, 30.0),
                'Gamma' : (30.1, (SAMPLE_RATE >> 1) - 1),}
# Coefficients pour filtres Butterworth numérique d'ordre N pour le filtrage passe-bande
bands_coeff  = {band : butter_bandpass(low, high, SAMPLE_RATE) for band, (low, high) in eeg_bands.items()}
# Nombre dévènement à prédir
num_events   = range(len(hands_event))

LAG, SCOPE

(-62, 500)

#### • Acquisition des données d'entrainement

In [3]:
### • Acquisition des données d'entrainement

target  = "../data/data.zip"
count   = len('train/')
fics    = [x[count :] for x in files_in_zip(target, directory = 'train')]

[fics.remove(x) for x in fics[:: -3]]

# Acquisition des fichiers du répertoir dans le fichier zip
train_csv = csv_in_zip(target, directory = 'train', files = fics)
label_csv = csv_in_zip(target, directory = 'y_train_only', files = fics)

notes   = filename(fics)
headers = [f"{t} . {i + 1}" for i, t in enumerate(notes)]
count   = range(len(train_csv))

# fics    = [f'B0{i}0{j}T.csv' for i in range(1, 9) for j in range(1, 4)]
# df_train_pkl = pkl_in_zip(path, fichier_specifique = 'epoched_train.pkl')

#### • Acquisition des données de test

In [ ]:
### • Acquisition des données de test

test_csv = csv_in_zip(target, directory = 'test')

#### • Catch22 émulateur

https://r-packages.io/packages/Rcatch22

In [ ]:
def splinefit(y : Vector, nBreaks : int = 3, deg : int = 3) -> Vector :
    size      : int = len(y)
    piecesExt : int = 4
    nSpline   : int = 4
    pieces    : int = 2
    nCoeff    : int = nSpline * piecesExt

    breaks = [0, int(size >> 1) - 1, size - 1]
    
    # -- splinebase
    
    # repeat spacing
    hCopy = list(adjacent(breaks)) * 2
    # add breaks
    hExt  = adjacent(list(np.flip(breaks[0] - np.cumsum(hCopy[1 :]))) +
                     list(breaks) +
                     list(breaks[2] + np.cumsum(hCopy[: deg])))
    # expand h using the index matrix ii
    I2 = [[min(j + i, piecesExt - 1) for j in range(piecesExt)]
          for i in range(deg + 1)]
    # expanded h
    H  = [hExt[I2[i % nSpline][i // nSpline]] for i in range(nCoeff)]
    # recursive generation of B-splines
    Q  = np.zeros((nSpline, piecesExt))
    # initialise polynomial coefficients + 1
    Kf = np.zeros((nCoeff, nSpline))
    # Q2 = np.zeros((nCoeff, nSpline)) 

    for i in range(0, nCoeff, nSpline) : Kf[i][0] = 1

    for k in range(1, nSpline) :
        # antiderivatives of splines
        for j in range(k) :
            for l in range(nCoeff) : Kf[l][j] *= H[l] / (k - j)
        
        for l in range(nCoeff) :
            for m in range(nSpline) : Q[l % nSpline][l // nSpline] += Kf[l][m]

            # Q2[l % nSpline][l // nSpline] += sum(K[l])

            # tmp = Q[l % nSpline][l // nSpline] - Q2[l % nSpline][l // nSpline]

            # if (tmp != 0) :
            #     inc += 1
            #     print(inc, k, l, tmp)

                # for m in range(nSpline) : print(K[l][m])
        
        # cumsum
        for l in range(piecesExt) :
            for m in range(1, nSpline) : Q[m][l] += Q[m - 1][l]
        
        for l in range(nCoeff) :
            md = l % nSpline 
            
            Kf[l][k] = 0 if (md == 0) else Q[md - 1][l // nSpline]  # questionable
        
        # normalise antiderivatives by max value
        fmax = [Q[nSpline - 1][i] for _ in range(nSpline) for i in range(piecesExt)]
        
        for j in range(k + 1) :
            for l in range(nCoeff) : Kf[l][j] /= fmax[l]

        # diff to adjacent antiderivatives
        for i in range(nCoeff - deg) :
            for j in range(k + 1) : Kf[i][j] -= Kf[deg + i][j]

        for i in range(1, nCoeff, nSpline) : Kf[i][k] = 0
    
    # scale coefficients
    scale = np.ones(nCoeff)
    
    for k in range(nSpline - 1) :
        scale = np.divide(scale, H)

        for i in range(nCoeff) :
            Kf[i][(nSpline - 1) - (k + 1)] *= scale[i]
    
    # reduce pieces and sort coefficients by interval number
    jj = [[nSpline * (j + 1) if (i == 0) else deg for j in range(pieces)]
          for i in range(nSpline)]
    
    for i in range(1, nSpline) :
        for j in range(pieces) : jj[i][j] += jj[i - 1][j]
    
    print([jj[i % nSpline][i // nSpline] - 2 for i in range(nSpline * pieces)])

    coefs_out = [[Kf[jj[i % nSpline][i // nSpline] - 2][j] for j in range(nSpline)]
                 for i in range(nSpline * pieces)]
    
    # -- create first B-splines to feed into optimization
    
    score : int = size * nSpline

    # x-values for B-splines
    xsB    = np.zeros(score)
    indexB = np.zeros(score)
    
    stop : int = 1

    for i in range(size) :
        if((i >= breaks[stop]) & (stop < nBreaks - 1)) : stop += 1
        
        m = stop - 1

        for j in range(nSpline) :
            p = i * nSpline + j

            xsB[p]    = i - breaks[m]
            indexB[p] = m * nSpline + j

    vB = [coefs_out[indexB[i]][0] for i in range(score)]
    
    for i in range(1, nSpline) :
        for j in range(score) :
            vB[j] = vB[j] * xsB[j] + coefs_out[indexB[j]][i]
    
    A = np.zeros((nSpline + 1) * size)

    stop = 0

    for i in range(score) :
        if (i / nSpline >= breaks[1]) : stop = 1

        A[(i % nSpline) + stop + (i // nSpline) * (nSpline + 1)] = vB[i]
    
    # coeffs of B-splines to combine by optimised weighting in x
    C = np.zeros((pieces + nSpline - 1, nSpline * pieces))

    n2 : int = nSpline << 1
    
    for i in range((nSpline ** 2) * pieces) :
        j = i // nSpline
    
        C[(i % nSpline) + (j % 2)][j] = coefs_out[i % n2][i // n2]
    
    x = np.linalg.solve(A, y) # lsqsolve_sub(nSpline + 1, A, y)
    
    # final coefficients
    coefsSpline = np.zeros((pieces, nSpline))
    
    # multiply with x
    for j in range(nSpline * pieces) :
        for i in range(nSpline + 1) :
            coefsSpline[j % pieces][j // pieces] += C[i][j] * x[i]
    
    # compute piecewise polynomial
    
    yOut = [coefsSpline[boolInt(i < breaks[1])][0] for i in range(size)]

    for i in range(1, nSpline) :
        for j in range(size) :
            p = boolInt(j < breaks[1])
            yOut[j] *= (j - breaks[1] * p) + coefsSpline[p][i]
    
    return yOut


In [20]:
def CO_Embed2_Dist_tau_d_expfit_meandiff(y : Vector) -> float :
    size : int = len(y)
    tau  : int = co_firstzero(y, size)
    
    if (tau > size / 10) : tau = int(size / 10)

    n : int = size - tau
    
    d = np.zeros(n)

    for i in range(n - 1) :
        d[i] = math.sqrt((y[i + 1] - y[i]) ** 2 + (y[i + tau] - y[i + tau + 1]) ** 2)
        
        if (math.isnan(d[i])) : return math.nan

    n_bins : int = num_bins_auto(d)

    if (n_bins == 0) : return 0
    
    hist_counts, bin_edges = histcounts_preallocated(d, n_bins)
    
    # normalise to probability
    histCountsNorm = hist_counts / (n - 1)
    
    # mean for exponential fit
    mn = np.mean(d)
     
    d_exp_fit_diff = np.zeros(n_bins)

    for i in range(n_bins) :
        expf = math.exp((bin_edges[i] + bin_edges[i + 1]) * -0.5 / mn) / mn

        if (expf < 0) : expf = 0
        
        d_exp_fit_diff[i] = abs(histCountsNorm[i] - expf)
    
    return np.mean(d_exp_fit_diff)

def CO_f1ecac(y : LComplex) -> float :
    # Compute autocorrelations
    corrs  = co_autocorrs(y)
    # Threshold to cross
    thresh = 1.0 / math.e

    for i in range(len(y)) :
        if (corrs[i + 1] < thresh) :
            return i + (thresh - corrs[i]) / (corrs[i + 1] - corrs[i])
    
    return len(y) - 1

def CO_FirstMin_ac(y : LComplex) -> int :
    size : int = len(y)
    corrs = co_autocorrs(y)

    for i in range(1, size - 1) :
        if ((corrs[i] < corrs[i - 1]) & (corrs[i] < corrs[i + 1])) :
            return i
    
    return size - 1

def CO_HistogramAMI_even_2_5(y : Vector, numBins : int = 5) -> float :
    tau : int = 2
    # set bin edges
    minValue = min(y)
    binStep  = (max(y) - minValue + .2) / numBins

    
    binEdges = minValue + binStep * np.array(range(numBins + 1)) - .1
    binsExt : int = len(binEdges)

    n_bins = range(numBins)
    
    #  count histogram bin contents
    bins1 = histBinAssign(y[: -tau], binEdges)
    bins2 = histBinAssign(y[tau :], binEdges)
    # joint
    bins12 = (np.array(bins1) - 1) * binsExt + bins2
    # fancy solution for joint histogram here
    jointHistLinear = np.histogram_bin_edges(bins12, range(1, binsExt ** 2 + 1), binsExt ** 2)
    # histcount_edges(bins12, binEdges12);
    
    # transfer to 2D histogram (no last bin, as in original implementation)
    pij = [[jointHistLinear[i * binsExt + j] for j in n_bins] for i in n_bins]
    
    sumBins = sum([sum(p) for p in pij])
    # normalise
    pij = [p / sumBins for p in pij]

     # marginals
    pi = np.zeros(numBins)
    pj = np.zeros(numBins)

    for i in n_bins :
        for j in n_bins :
            pi[i] += pij[i][j]
            pj[j] += pij[i][j]

    # mutual information
    ami : float = 0

    for i in n_bins :
        for j in n_bins :
            if(pij[i][j] > 0) :
                ami += pij[i][j] * math.log(pij[i][j] / (pj[j] * pi[i]))
    
    return ami

def CO_trev_1_num(y : Vector) -> float :
    return np.mean(adjacent(y) ** 3)

def DN_HistogramMode(y : Vector, nBins : int = 10) -> float :
    numMaxs : int = 1
    maxCount = 0
    out = 0
    
    histCounts, binEdges = histcounts_preallocated(y, nBins)
    
    for i in range(nBins) :
        if (histCounts[i] > maxCount) :
            maxCount = histCounts[i]
            numMaxs  = 1
            out      = (binEdges[i] + binEdges[i + 1])
        elif (histCounts[i] == maxCount) :
            numMaxs += 1
            out     += (binEdges[i] + binEdges[i + 1])
        
    return out / (numMaxs << 1)

def DN_HistogramMode_10(y : Vector) -> float :
    return DN_HistogramMode(y, 10)

def DN_HistogramMode_5(y : Vector) -> float :
    return DN_HistogramMode(y, 10)

def DN_Mean(a : Vector) -> float :
    return a.mean()

def DN_OutlierInclude_np_001_mdrmd(y : Vector, sign : int) -> float :
    size : int   = len(y)
    tot  : int   = 0
    inc  : float = 1e-2 # 0.01
    yWork = []

    constant_flag : bool = True

    # apply sign and check constant time series
    for x in y :
        if (x != y[0]) : constant_flag = False

        # apply sign, save in new variable
        yWork.append(sign * x)
        
        # count pos / negs
        if (yWork[-1] >= 0) : tot += 1

    if (constant_flag) : return 0
    
    # find maximum (or minimum, depending on sign)
    max_val = max(yWork)
    
    #  maximum value too small ? return 0
    if (max_val < inc) : return 0
    
    # save the indices where y > threshold
    r = []
    #  save the median over indices with absolute value > threshold
    msDti1 = []
    msDti3 = []
    msDti4 = []
    
    k = 100 / tot
    s = 2 / size
    
    nThresh = int(max_val / inc) + 1
    
    for j in range(nThresh) :
        r += [i + 1 for i in range(size) if (yWork[i] >= j * inc)]
        
        #  intervals between high-values
        tmp = adjacent(r)
        high_size = len(tmp)
        
        msDti1.append(np.mean(tmp[: high_size]))
        msDti3.append(k * high_size)
        msDti4.append(np.median(r) * s - 1)
    
    mj      : int = 0
    trimthr : int = 2
    fbi     : int = nThresh - 1

    for i in range(nThresh) :
        if (msDti3[i] > trimthr) : mj = i

        k = nThresh - i - 1

        if (math.isnan(msDti1[k])) : fbi = k
    
    return np.median(msDti4[: (mj if mj < fbi else fbi) + 1])

def DN_OutlierInclude_p_001_mdrmd(y : Vector) -> float :
    return DN_OutlierInclude_np_001_mdrmd(y, 1.0)

def DN_OutlierInclude_n_001_mdrmd(y : Vector) -> float :
    return DN_OutlierInclude_np_001_mdrmd(y, -1.0)

def DN_Spread_Std(a : Vector) -> float :
    return np.std(a)

def FC_LocalSimple_mean_tauresrat(y : Vector, train_length : int) -> float :
    size : int = len(y)
    
    res       = fc_local_simple_mean(y, train_length)
    resAC1stZ = co_firstzero(res, size - train_length)
    yAC1stZ   = co_firstzero(y, size)
    
    return resAC1stZ / yAC1stZ

def FC_LocalSimple_mean_stderr(y : Vector, train_length : int) -> float :
    return np.std(fc_local_simple_mean(y, train_length))

def FC_LocalSimple_mean3_stderr(y : Vector) -> float :
    return FC_LocalSimple_mean_stderr(y, 3)

def FC_LocalSimple_mean1_tauresrat(y : Vector) -> float :
    return FC_LocalSimple_mean_tauresrat(y, 1)

def IN_AutoMutualInfoStats_40_gaussian_fmmi(y : Vector, tau : int = 40) -> float :
    size : int = len(y)
    # don't go above half the signal length
    tau = min(tau, math.ceil(size / 2))
    # compute autocorrelations and compute automutual information
    ami = [math.log(1 - autocorr_lag(y, i + 1) ** 2) * -.5 for i in range(tau)]

    # find first minimum of automutual information
    for i in range(1, tau - 1) :
        if((ami[i] < ami[i - 1]) & (ami[i] < ami[i + 1])) : return i

    return tau

def MD_hrv_classic_pnn40(y : Vector, pNNx : int = 40) -> float :
    return len([0 for x in adjacent(y) if abs(x * 1000) > pNNx]) / len(y)

@deprecated("Problème dans l'implémentation de 'splinefit'")
def PD_PeriodicityWang_th0_01(y : Vector, th : float = 1e-2) -> int :
    size  : int = len(y)
    # compute autocorrelations up to 1/3 of the length of the time series
    acmax : int = int(math.ceil(size / 3.0))
    
    # fit a spline with 3 nodes to the data
    # subtract spline from data to remove trend
    ySub = np.array(y) - splinefit(y)
    # correlation/ covariance the same, don't care for scaling
    # (cov would be more efficient)
    acf = [cov_mean(ySub[: -tau], ySub[tau :]) for tau in range(1, acmax)]

    # find troughts and peaks
    troughs = []
    peaks   = []

    for i in range(1, acmax - 1) :
        slopeIn  = acf[i] - acf[i - 1]
        slopeOut = acf[i + 1] - acf[i]
        
        if   ((slopeIn < 0) & (slopeOut > 0)) : troughs.append(i)
        elif ((slopeOut < 0) & (slopeIn > 0)) : peaks.append(i)
    
    # search through all peaks for one that meets the conditions:
    # (a) a trough before it
    # (b) difference between peak and trough is at least 0.01
    # (c) peak corresponds to positive correlation
    nTroughs : int = len(troughs)
    
    for i in range(len(peaks)) :
        ip : int = peaks[i]
        # find trough before this peak
        j  : int = -1

        while ((troughs[j + 1] < ip) & (j + 1 < nTroughs)) : j += 1

        # (a) should be implicit
        if (j == -1) : continue
        # (b) different between peak and trough it as least 0.01
        if (acf[ip] - acf[troughs[j]] < th) : continue
        # (c) peak corresponds to positive correlation
        if (acf[ip] < 0) : continue
        
        # use this frequency that first fulfils all conditions.
        return ip
    
    return 0

def SB_BinaryStats_diff_longstretch0(y : Vector) -> int :
    return sb_binary_stats_diff_longstretch(adjacent(y) < 0)

def SB_BinaryStats_mean_longstretch1(y : Vector) -> int :
    return sb_binary_stats_diff_longstretch(deviation(y) <= 0)

def SB_MotifThree_quantile_hh(y : Vector, alphabet_size = 3) -> float :
    size : int = len(y)
    inc  : int = size - 1
    # transfer to alphabet
    yt   = sb_coarsegrain(y, "quantile", 3)
    abcd = range(alphabet_size) 

    # using selfresizing array for memory efficiency. Time complexity
    # should be comparable due to ammotization.
    Rh = [[j for j in range(size) if (yt[j] == i + 1)] for i in abcd]
    
    # removing last item if it is == max possible idx since later
    # we are taking idx + 1 from yt
    for i in abcd :
        l = len(Rh[i])

        if ((l != 0) & (Rh[i][l - 1] == inc)) : Rh[i] = Rh[i][: l - 1]

    # allocate separately
    out2 = [[len([k for k in Rh[i] if yt[k + 1] == (j + 1)]) / inc for j in abcd]
           for i in abcd]

    return sum([f_entropy(x) for x in out2])

def SB_TransitionMatrix_3ac_sumdiagcov(y : Vector, num_groups : int = 3) -> float :
    size  : int = len(y)
    tau   : int = co_firstzero(y, size)
    nDown : int = int((size - 1) / tau) + 1

    grp   = range(num_groups)
    yDown =  [y[i * tau] for i in range(nDown)]
    # transfer to alphabet
    yCG = sb_coarsegrain(yDown, "quantile", num_groups)
    T   = np.zeros((num_groups, num_groups))
    
    # more efficient way of doing the below 
    for j in range(nDown - 1) : T[yCG[j] - 1][yCG[j + 1] - 1] += 1
    
    cov_tmp  = 0
    dev_cols = [deviation([T[i][j] / (nDown - 1) for i in grp]) for j in grp]

    for i in grp :
        for j in range(i, i + 1) : cov_tmp += np.dot(dev_cols[i], dev_cols[j])

    """
    COV = np.zeros((num_groups, num_groups))
    
    for i in range(num_groups) :
        for j in range(i, num_groups) :
            covTemp = np.dot(dev_cols[i], dev_cols[j]) / (num_groups - 1)
            
            COV[i][j] = covTemp
            COV[j][i] = covTemp

    print([COV[i][i] for i in range(num_groups)])
    
    return sum([COV[i][i] for i in range(num_groups)])
    """

    return cov_tmp / (num_groups - 1)

def SC_FluctAnal_2_50_1_logi_prop_r1(y : Vector, lag : int, how : str) -> float :
    size : int = len(y)
    
    # generate log spaced tau vector
    lin_low  = math.log(5)
    lin_high = math.log(size / 2)
    
    nTauSteps : int = 50
    tauStep = (lin_high - lin_low) / (nTauSteps - 1)

    tau = [int(round(math.exp(lin_low + i * tauStep))) for i in range(nTauSteps)]
    
    # check for uniqueness, use ascending order
    nTau : int = nTauSteps

    for i in range(nTauSteps - 1) :
        while ((tau[i] == tau[i + 1]) & (i < nTau - 1)) :
            tau[i + 1 : nTauSteps - 1] = tau[i + 2 : nTauSteps]
            # for j in range(i + 1, nTauSteps - 1) : tau[j] = tau[j + 1]

            # lost one
            nTau -= 1
    
    # fewer than 12 points -> leave.
    if(nTau < 12) : return 0
    
    sizeCS : int = int(size / lag)
    yCS = [0 for _ in range(sizeCS)] # malloc(sizeCS * sizeof(double))

    # transform input vector to cumsum
    yCS[0] = y[0]

    for i in range(sizeCS - 1) : yCS[i + 1] = yCS[i] + y[(i + 1) * lag]
    
    # first generate a support for regression (detrending)
    xReg = range(1, tau[nTau - 1] + 1)
    
    # iterate over taus, cut signal, detrend and save amplitude of remaining signal
    F = np.zeros(nTau)

    for i in range(nTau) :
        nBuffer : int = int(sizeCS / tau[i])

        for j in range(nBuffer) :
            p = j * tau[i]

            _, m, b = linreg(xReg, yCS[p :], tau[i])
            
            buffer = [yCS[p + k] - (m * (k + 1) + b) for k in range(tau[i])]
            
            if (how == "rsrangefit") :
                F[i] = (max(max(buffer), tau[i]) - min(min(buffer), tau[i])) ** 2
            elif (how == "dfa") :
                F[i] = sum(np.array(buffer[: tau[i]]) ** 2)
            else :
                return 0.0
        
        if   (how == "rsrangefit") :
            F[i] = math.sqrt(F[i] / nBuffer)
        elif (how == "dfa") :
            F[i] = math.sqrt(F[i] / (nBuffer * tau[i]))

    logtt = np.log(tau[: nTau])
    logFF = np.log(F[: nTau])

    minPoints : int = 6

    sserr = np.zeros(nTau - 2 * minPoints + 1)

    for i in range(minPoints, nTau - minPoints + 1) :
        # this could be done with less variables of course    
        it = i - 1
        
        _ , m1, b1 = linreg(logtt, logFF, i)
        _ , m2, b2 = linreg(logtt[it :], logFF[it :], nTau - it)
        
        # buffer = [logtt[j] * m1 + b1 - logFF[j] for j in range(i)]
        buffer = logtt[: i] * m1 + b1 - logFF[: i]
        
        sserr[i - minPoints] += np.linalg.norm(buffer[: i])

        buffer = [logtt[j + it] * m2 + b2 - logFF[j + it] for j in range(nTau - it)]
        
        sserr[i - minPoints] += np.linalg.norm(buffer[: nTau - it])
    
    minimum = np.min(sserr)

    return ((np.where(sserr == minimum)[0] + 1) / nTau)[0]

def SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1(y : Vector) -> int :
    return SC_FluctAnal_2_50_1_logi_prop_r1(y, 2, "dfa")

def SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1(y : Vector) -> int :
    return SC_FluctAnal_2_50_1_logi_prop_r1(y, 1, "rsrangefit")

def SP_Summaries_welch_rect(y : Vector, what : str) -> float :
    size : int = len(y)
    nfft : int = pow2(size)
    Fs = 1.0    # sampling frequency
    
    # rectangular window for Welch-spectrum / compute Welch-power
    nWelch, S, f = welch(y, nfft, Fs, np.ones(size))
    
    # angualr frequency and spectrum on that
    # w  = f * 2 * math.pi
    Sw = S / (2 * math.pi)
    
    for s in Sw :
        if (math.isinf(s)) : return 0
    
    csS = np.cumsum(Sw)
    
    if (what == "centroid") :        
        csSThres = csS[nWelch - 1] * 0.5

        for i in range(nWelch) :
            if(csS[i] > csSThres) : return f[i] * 2 * math.pi
    elif (what == "area_5_1") :
        return (sum(Sw[: int(nWelch // 5)]) * (f[1] - f[0]) * 2 * math.pi).real
    
    return 0

def SP_Summaries_welch_rect_area_5_1(y : Vector) -> float :
    return SP_Summaries_welch_rect(y, "area_5_1")
    
def SP_Summaries_welch_rect_centroid(y : Vector) -> float :
    return SP_Summaries_welch_rect(y, "centroid")

In [21]:
catch22 = [CO_Embed2_Dist_tau_d_expfit_meandiff,
          CO_f1ecac,
          CO_FirstMin_ac,
          CO_HistogramAMI_even_2_5,
          CO_trev_1_num,
          DN_HistogramMode_10,
          DN_HistogramMode_5,
          DN_Mean,
          DN_OutlierInclude_p_001_mdrmd,
          DN_OutlierInclude_n_001_mdrmd,
          DN_Spread_Std,
          FC_LocalSimple_mean3_stderr,
          FC_LocalSimple_mean1_tauresrat,
          IN_AutoMutualInfoStats_40_gaussian_fmmi,
          MD_hrv_classic_pnn40,
        #   PD_PeriodicityWang_th0_01,
          SB_BinaryStats_diff_longstretch0,
          SB_BinaryStats_mean_longstretch1,
          SB_MotifThree_quantile_hh,
          SB_TransitionMatrix_3ac_sumdiagcov,
          SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,
          SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,
          SP_Summaries_welch_rect_area_5_1,
          SP_Summaries_welch_rect_centroid,]

In [23]:
# print(np.histogram_bin_edges(trains['C3'][0], 5) - .1, min(trains['C3'][0]) - .1, max(trains['C3'][0]) + .1)
 #, trains['C3'][0] #, 

# MD_hrv_classic_pnn40(trains['C3'][0], 1525)

# SB_MotifThree_quantile_hh(trains['C3'][0])

# DN_OutlierInclude_np_001_mdrmd(trains['C4'][0], 1), rsrangefit
# SC_FluctAnal_2_50_1_logi_prop_r1(trains['C4'][5], 2, "dfa")
# SP_Summaries_welch_rect(trains['C4'][5], "area_5_1")
# stddev(trains['C3'][0], trains['C4'][3])

# CO_Embed2_Dist_tau_d_expfit_meandiff(trains['C3'][57])

# np.complex128(deviation(trains['C3'][107]))

[foo(trains['C4'][107]) for foo in catch22]

# co_autocorrs(trains['C3'][107])

# splinefit(trains['C3'][0])


C:\Users\b.luron\AppData\Local\Temp\ipykernel_34856\1059410348.py:467: ComplexWarning: Casting complex values to real discards the imaginary part
  if (math.isinf(s)) : return 0


[0.06835508033172982,
 0.5605481639257237,
 1,
 0.0038449298910453204,
 -0.07952272775049883,
 -0.9999237048905165,
 -0.9999237048905165,
 -1.1932555123216602,
 0.6120000000000001,
 -0.17599999999999993,
 3.476309819282354,
 2.4230646499380937,
 1.0,
 5,
 0.982,
 9,
 43,
 1.7882225782083592,
 0.0414175043473721,
 0.717391304347826,
 0.043478260869565216,
 0.009029671206882637,
 1.5830681730979816]

#### • Pré-traitement des données

In [4]:
### • Pré-traitement des données

def data_spliting(datas : list[Board], Channels : Clause, number_event : int | Index,
                  labels : list[Board] | None = None, merge : bool = False, level : bool = True) :
    count = range(len(datas))
    parts = []        #
    temp  = [[], []]  # Les époques pour tous les cannaux et tous les évènements.
    spots = [[], []]  # Apparitions des évènements
    # Pour la standardisation du nombre d'échantillon max conservé
    ceil  = min([len(labels[i]['EventType']) for i in count])

    if type(number_event) == int : number_event = range(number_event)

    # Extraction des données relavitives à l'apparition des évènements.
    if level :
        for i in count :
            df   = datas[i]
            kind = labels[i]['EventType'][: ceil]
            loc  = np.where(df['EventStart'] == 1)[: ceil]

            parts.append(zero_removal(df['C3'], 75))

            for j in number_event :
                spots[j].append(np.array(*loc)[*np.where(kind == j)])

                # [tf.convert_to_tensor(X) for X in event_epochs(spots[j][-1], SCOPE, LAG)]
                room = event_epochs(spots[j][-1], SCOPE, LAG)

                temp[j].append([full_event(df[c], room, merge) for c in Channels])
    else :
        for i in count :
            df   = datas[i]
            kind = label_csv[i]['EventType']
            loc  = np.where(df['EventStart'] == 1)

            parts.append(zero_removal(df['C3'], 75))

            for j in number_event :
                spots[j].append(np.array(*loc)[*np.where(kind == j)])

                room = event_epochs(spots[j][-1], SCOPE, LAG)

                temp[j].append([full_event(df[c], room, merge) for c in Channels])

    # Regroupement des données en fonction du type de l'évènement et du cannal d'observation
    if merge :
        temp = [[[np.append([], T[j :: 3]) for T in temp[i]] for j in range(3)] for i in number_event]
    else :
        store = [[[], [], []], [[], [], []]]
        
        [[[[store[i][j].append(G) for G in X] for j, X in enumerate(T)] for T in temp[i]] for i in number_event]

        temp = store

    eras = [pd.DataFrame({**dict(zip(Channels, [pd.Series(X) for X in temp[i]])), 'EventType': i}) for i in number_event]

    return eras, spots, parts

# %time
train_eras, train_spots, train_parts = data_spliting(train_csv, eeg_Chans, num_events, label_csv)

# train_samples = samples(len(train_eras[0]))
trains        = pd.concat(train_eras, ignore_index = True)

In [5]:
# DN_OutlierInclude_np_001_mdrmd(trains['C3'][0], 1)
SB_BinaryStats_mean_longstretch1(trains['C3'][1])
# np.array([1, 2, 3, 4]) + [5, 6, 7, 8]

NameError: name 'SB_BinaryStats_mean_longstretch1' is not defined

In [ ]:
def catch(data : Board, col : str, channels : Clause, event : int,
            norm : bool = False) -> Board :
    func = [np.min, np.max, np.ptp, np.std, np.var, np.mean, np.median, np.average]
    tmp  = data[data[col] == event]
    name = [f.__name__ for f in func]
    head = np.append([f"{c}_{f}" for c in channels for f in name],
                     [f"{f}_diff" for f in name])

    if norm :
        tmp = [[normalized(x) for x in tmp[c]] for c in channels]

        _c3, _c4, _cz = [[[f(v) for v in s] for f in func] for s in tmp]
    else :
        _c3, _c4, _cz = [[[f(v) for v in tmp[c]] for f in func] for c in channels]
    
    _ff = [np.array(v1) - v0 for v0, v1 in zip(_c3, _c4)]
    _df = pd.DataFrame(np.array((*_c3, *_c4, *_cz, *_ff)).T, columns = head)
    # _df = pd.DataFrame(np.array((*_cz, *_ff, )).T)
    
    _df[col] = event

    return _df

norm : bool = False

df_left  = catch(trains, 'EventType', eeg_Chans, 0, norm = norm)
df_right = catch(trains, 'EventType', eeg_Chans, 1, norm = norm)

X = pd.concat((df_left, df_right), ignore_index = True)
y = X['EventType']

display(X)

X.drop(columns = ['EventType'], inplace = True)

# tmp = [[normalized(x) for x in trains[c]] for c in eeg_Chans]

# display(np.shape(tmp))

# tmp = pd.DataFrame((tmp), index = eeg_Chans).T

# func    = [np.min, np.max, np.ptp, np.std, np.var, np.mean, np.median, np.average], columns = eeg_Chans

# tmp     = trains[trains['EventType'] == 0]
# left_c3 = [[f(v) for v in tmp['C3']] for f in func]
# left_c4 = [[f(v) for v in tmp['C4']] for f in func]
# left_cz = [[f(v) for v in tmp['Cz']] for f in func]
# left_ff = [np.array(v1) - v0 for v0, v1 in zip(left_c3, left_c4)]
# left_cr = np.correlate(tmp['C3'].values, tmp['C4'].values)

# tmp      = trains[trains['EventType'] == 1]
# right_c3 = [[f(v) for v in tmp['C3']] for f in func]
# right_c4 = [[f(v) for v in tmp['C4']] for f in func]
# right_cz = [[f(v) for v in tmp['Cz']] for f in func]
# right_ff = [np.array(v1) - v0 for v0, v1 in zip(left_c3, left_c4)]
# right_cr = np.correlate(tmp['C3'].values, tmp['C4'].values)

# df_left  = pd.DataFrame(np.array((*left_c3, *left_c4, *left_ff, *left_cz)).T)
# df_right = pd.DataFrame(np.array((*right_c3, *right_c4, *right_ff, *right_cz)).T)

# df_left['EventType']  = 0
# df_right['EventType'] = 1

# len(left_cr[0]), np.shape(left_c3), np.shape(tmp['C3'].values)

#### • Test prédiction

In [ ]:
### • Visualisation des spectrogrammes / Test

def logMelSpectrogram(data : Vector, rate : int, dt : float = 1e-2) -> Vector :
    tps = 1 << int(np.floor(np.log2(rate * dt)))
    # print(tps)
    # Spectrogramme
    stfts = np.abs(librosa.stft(y = data, n_fft = tps, hop_length = 1 << 2, center = True)).T
    # Filtre de MEL
    liny  = librosa.filters.mel(sr = rate, n_fft = tps + 1, n_mels = stfts.shape[-1]).T
    # Application du filtre au spectrogramme
    mel_  = np.tensordot(stfts, liny, 1)

    return np.log(mel_ + 1e-6)
    
def structure(data : Board | Vector, rate : int, whr : Clause) -> Vector :
    # return np.array([logMelSpectrogram(X, rate, 2) for X in data[whr]])
    # return np.stack([[signal.welch(X, rate)[1] for X in data[c]] for c in whr], axis = 2)
    return np.stack(trains['C4'] - trains['C3'], axis = 1)
    # return np.stack([[X for X in data[c]] for c in whr], axis = 2)

def img_spectrogram(raw : Vector, rate : int, nfft : int = 1 << 10) -> Vector :
    return librosa.feature.melspectrogram(y = raw, sr = rate, hop_length = 1, 
                            n_fft = nfft, n_mels = 32, fmin = 0, fmax = 20, win_length = 32)

def spectrogram_dep(data : Board, rate : int, channels : Clause, n_row : int = 5, n_col : int = 12) :
    sample = np.random.default_rng().integers(data.shape[0], size = n_row)

    sample.sort()

    plt.figure(figsize = (18, 2 * .48 * n_row))

    pos = 0

    for k in sample :
        for c in channels :
            # x   = normalized(data[c][k])
            raw = img_spectrogram(raw = data[c][k], rate = rate)
            pos += 1
            
            plt.subplot(n_row, n_col, pos)
            plt.title(f"{((pos - 1) // 3) + 1} . {k} - {c}", fontsize = 8)
            librosa.display.specshow(data = 1 - raw, sr = rate, hop_length = 1)
            
            # pos = n_col * (i >> 1) + j
            # f, t, Sxx = signal.spectrogram(x, rate)
            # plt.subplot(n_row, n_col, pos + 4)
            # plt.pcolormesh(t, f, 1 - Sxx, shading = 'gouraud')

    plt.tight_layout()
    plt.show();

def spectrogram(data : Board, rate : int, channels : Clause, n_row : int = 5, n_col : int = 12) :
    sample = np.random.default_rng().integers(data.shape[0], size = n_row)

    sample.sort()

    plt.figure(figsize = (18, 2 * .48 * n_row))

    freq = np.arange(1, rate >> 1)
    pos  = 0
    extd = np.append([0, 1, 1], freq[-1])
    
    for k in sample :
        for c in channels :
            pos += 1
            x   = normalized(data[c][k])
            coefficients, frequencies = pywt.cwt(x, scales = freq, wavelet = 'cmor')

            plt.subplot(n_row, n_col, pos)
            plt.imshow(np.abs(coefficients), aspect = 'auto', cmap = 'jet') #, extent = extd
            # plt.colorbar(label="Magnitude")
            # plt.ylabel("Scale")
            # plt.xlabel("Time")
            # plt.title("CWT of a Chirp Signal")
            plt.axis('off')

    plt.show()

def plot_logMelSpectrogram(data, rate) :
    sns.heatmap(np.rot90(logMelSpectrogram(data, rate)), cmap = 'inferno')
    
    # loc, _ = plt.xticks()
    # l      = np.round((loc - loc.min()) * len(data) / fe / loc.max(), 2), vmin = -6

    # plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")

In [ ]:
coefficients, frequencies = pywt.cwt(train_eras[0]['C3'][0], scales = np.arange(1, SAMPLE_RATE >> 1), wavelet = 'cmor')

1 / frequencies

In [ ]:
H = [list(harmonic(x, bands_coeff).values()) for x in [trains[c] for c in eeg_Chans]]

In [ ]:
np.shape(H), np.shape(np.stack(np.stack(H, axis = 1), axis = 2)), np.shape(H[0][1][0])
# harmonic(trains['C3'][256], bands_coeff).values())), H

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trains.drop(columns = ['EventType']),
                                                    trains['EventType'], test_size = .2, random_state = 42)

# X_train, X_test, y_train, y_test = train_test_split([v.tolist() for v in np.array(trains[eeg_Channels])],
#                                                     trains['EventType'], test_size = .2, random_state = 42)

In [ ]:
# train_dataset = np.array(X_train)
# test_dataset  = X_test

# train_dataset = structure(X_train, SAMPLE_RATE, eeg_Chans[:2])
# test_dataset  = structure(X_test, SAMPLE_RATE, eeg_Chans[:2])

train_dataset = structure(X_train, SAMPLE_RATE, eeg_Chans[:2])
test_dataset  = structure(X_test, SAMPLE_RATE, eeg_Chans[:2])

# print(train_dataset.shape)

In [ ]:
#### • Test prédiction

# UNITS     : int   = 100
BATCHSIZE : int   = 30
EPOCH     : int   = 1000
ZERO      : int   = 64
DROPOUT   : float = .2
# kl_divergence mean_squared_logarithmic_error mean_absolute_error
LOSS      : None  = 'sparse_categorical_crossentropy'
ACTIV     : None  = LeakyReLU   # PReLU, 
OPTIMIZER : None  = 'AdamW'     # adamax, , adafactor, adam, nadam

model = Sequential()

# - 1 -
model.add(Conv1D(filters = ZERO, kernel_size = (5), dilation_rate = 2,
                 input_shape = train_dataset.shape[1: ]))
model.add(Dropout(DROPOUT))
model.add(ACTIV())

# - 2 -
model.add(Conv1D(filters = ZERO << 1, kernel_size = (5), dilation_rate = 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())

# - 3 -
model.add(Conv1D(filters = ZERO << 2, kernel_size = (5), dilation_rate = 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())
model.add(GlobalAveragePooling1D()) # 

# Classification
model.add(Dense(ZERO << 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())
model.add(Dense(len(hands_event), activation = 'softmax')) # sigmoid  

'''
# Ajout de la premiere couche lstm
model.add(LSTM(ZERO, input_shape = train_dataset.shape[1:], activation = ACTIV(), return_sequences = True)) #
model.add(LSTM(ZERO, dropout = DROPOUT, return_sequences = False))

# Ajout de la couche de sortie
model.add(Dense(len(hands_event), activation = 'softmax'))
'''

model.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = ['accuracy'])
# model.summary();

In [ ]:
stop    = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 50)
history = model.fit(train_dataset, y_train, validation_data = (test_dataset, y_test), verbose = 1,
                    batch_size = BATCHSIZE, epochs = EPOCH, callbacks = [stop])

In [ ]:
pred = model.predict(test_dataset)

sum([np.where(x > .5)[0][0] for x in pred] == y_test) / len(pred)

In [ ]:
history_dict = history.history
loss_values  = history_dict['loss']
acc_values   = history_dict['accuracy']
absc         = range(1, len(loss_values) + 1)

plt.figure(figsize = (12, 4))

plt.subplot(121)
plt.plot(absc, loss_values, label = 'Loss')
plt.plot(absc, acc_values, label = 'Accuracy')
plt.title('Training')
plt.xlabel('Epochs')
plt.legend()

plt.subplot(122)
plt.plot(absc, history_dict['val_loss'], label = 'Loss')
plt.plot(absc, history_dict['val_accuracy'], label = 'Accuracy')
plt.title('Testing')
plt.xlabel('Epochs')
plt.legend()
plt.show();

#### • Visualisation densité spectrale du Signal

In [ ]:
### • Visualisation Densité Spectral du Signal

plot_psd(train_csv, train_eras, rate = SAMPLE_RATE, Channels = eeg_Chans, titled = headers)

#### • Densité spectrale / échantillon

In [ ]:
### • Densité spectral / échantillon

inc   = 40
scp = samples(train_samples, inc)
boolInt = -2

plt.figure(figsize = (15, inc * 1.5))

for i in scp :
    boolInt += 2

    for c in eeg_Chans :
        x = train_eras[0][c][i]
        yest, Pxx_den = signal.welch(x, SAMPLE_RATE)   # , scaling = 'spectrum'
        
        plt.subplot(inc, 4, boolInt + 1)
        plt.semilogy(yest, Pxx_den, label = c)
        plt.title(f"welch - {i + 1}", fontsize = 11)
        plt.grid()

        plt.subplot(inc, 4, boolInt + 2)
        r, _ = plt.psd(x, Fs = SAMPLE_RATE, label = c) # , NFFT = NFFT
        plt.title(f"psd - {i + 1}", fontsize = 11)
        plt.xlabel('')
        plt.ylabel('')
        # plt.legend()

plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
plt.tight_layout()
plt.show();

# f, Pxx_den = signal.welch(train_eras[0]['C3'][752], SAMPLE_RATE)

# print(len(Pxx_den))

#### • Visualisation Epoques

In [ ]:
### • Visualisation Epoques

for i in range(len(fics))[:: 3] :
    plot_signal(train_csv[i], train_parts[i], train_spots[0][i], train_spots[1][i], channels = eeg_Chans, # 
                period = SCOPE, lag = LAG, title = headers[i])

#### • Visualisation décomposition des signaux

In [ ]:
### • Visualisation Décomposition des signaux

# Test de décomposition des signaux en bandes de fréquences spécifiques compatibles avec les répartitions usuelles
# dans le domaine des EEG ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']

for df, token in zip(train_eras, ['Gauche', 'Droite']) :
    print(f"Exemples - Évènement Discriminé Main {token}")
    plot_wavelets(df, bands_coeff, eeg_Chans, scope = 30, headers = headers)

#### • Visualisation des spectrogrammes (test)

In [ ]:
for i, t in zip(num_events, ['Gauche', 'Droite']) :
    print(f"Exemples - Évènement Discriminé Main {t}")
    spectrogram(train_eras[i], SAMPLE_RATE, eeg_Chans, 32)

#### • Test PCA - (Non cloncluant)

In [ ]:
### • Test PCA - (Non cloncluant)

nca = SCOPE >> 2
pca = PCA(nca)

_, ax = plt.subplots(nrows = 2, ncols = 3, figsize = (15, 5))

for i, d in enumerate(train_eras) :
    for j, c in enumerate(eeg_Chans) :
        Z = sc.fit_transform(list(d[c].to_list())) # 
        principal_components = pca.fit_transform(Z)
        
        ax[i, j].plot(range(nca), np.cumsum(pca.explained_variance_ratio_))
        ax[i, j].set_title(f'{c} . {i}')

plt.tight_layout()
plt.show();

#### • MNE époque (test)

In [ ]:
### • MNE époque (test)

raw_csv = train_csv[0][eeg_Chans]
info    = mne.create_info(ch_names = eeg_Chans, sfreq = SAMPLE_RATE, ch_types = 'eeg')
raw_mne = mne.io.RawArray(raw_csv.T * 1e-6, info)
loc     = np.where(train_csv[0]['EventStart'] == 1)[0]

# display(compare(np.sort(np.concatenate((train_spots[0][0], train_spots[1][0]))), loc))

tmin, tmax = -0., 1

# loc = mne.find_events(raw_mne, stim_channel = 'C3')
# event_id = dict(C3 = 1, aud_r = 2, vis_l = 3, vis_r = 4)
# raw = mne.io.Raw(raw_mne, preload = True)
# raw.filter(2, None, method = 'iir')           # replace baselining with high-pass
# events = mne.read_events(event_fname)

# raw.info['bads'] = ['MEG 2443']  # set bad channels
# picks = mne.pick_types(info, meg = 'grad', eeg = True, eog = False, exclude = 'bads')
# Read epochs
absc = mne.Epochs(raw_mne, np.array([loc, loc, loc]).T, None, tmin, tmax, proj = False,
                    picks = None, baseline = None, preload = True, verbose = False) # event_id picks

# labels = epochs.events[::5, -1]

# events

# raw_mne.plot();

# raw_mne['C3'][0][0], len(df_train_csv[2]['Cz'])

display(absc)

#### • Test de classification - Proposition inputs 01

In [ ]:
#### • Test de classification - Proposition inputs 01

# X1 = np.where(trains['EventType'] == 0, trains['C3'], trains['Cz']) 
# X2 = np.where(trains['EventType'] == 0, trains['Cz'], trains['C4']) 
# X = [signal.welch(list(v), SAMPLE_RATE)[1] for v in X1 + X2]

# X = [np.append([], list(harmonic(v, bands_coeff).values())) for v in X]
# X = np.where(trains['EventType'] == 0, trains['C3'], 2 * trains['Cz'])
# X = [signal.welch(list(v), SAMPLE_RATE)[1] for v in X], trains['C3'] + trains['Cz'], trains['C4'] + trains['Cz']

# diff = np.array([np.cos(x) for x in (trains['C3'] ** 2 + trains['C4'] ** 2)])  + 2 * trains['Cz']

# dist =np.array([list(np.sqrt(x)) for x in (trains['C3'] ** 2 + trains['C4'] ** 2)])
# display(np.shape(np.array()))

# diff = np.array([d - v for v, d in zip(, dist)]) - trains['Cz'] * [x.mean() for x in trains['Cz']]

# cz_min = [v.min() for v in trains['Cz']]
# cz_max = [v.max() for v in trains['Cz']]

# display(compare(cz_min, cz_max), )

# trio = zip(trains['C4'], trains['C3'], trains['Cz'])

# prd = [(c3 - c4) * cz for c3, c4, cz in trio] 

# 'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma'

# b, a = bands_coeff['Delta']

# c3 = np.array([bandpass_filter(bw, b, a) for bw in trains['C3']])
# c4 = np.array([bandpass_filter(bw, b, a) for bw in trains['C4']])

# diff = (c4 - c3) / trains['Cz'].max() #/ cz.max() # [for v in cz]
# X = [np.sign(s) for s in trains['C4'] - trains['C3']]
# X = np.array([normalized(bandpass_filter(bw, b, a)) for bw in X])

# X = [sk_p.minmax_scale(pywt.dwt(c4, wavelet = 'db4')[0] - pywt.dwt(c3, wavelet = 'db4')[0]) for c3, c4 in zip(trains['C3'], trains['C4'])]
# X = [v) for v in X]

# X = [pywt.dwt(v, wavelet = 'db4')[0] for v in X]
# X = [normalized(pywt.dwt(v, wavelet = 'db4')[0]) for v in X]
# X = [normalized(pywt.dwt(v, wavelet = 'db4')[0]) for v in X]
# X = [normalized(v) for v in X]

# X = [np.append([], v.tolist()) for v in X]

# ret = pywt.dwt(data, wavelet = 'db1') #, level = 4, mode = 'antisymmetric'

# for o in ret :
#     print(np.shape(o))

# # np.shape(ret[3])[0] / np.shape(train_csv[0]['Cz'])[0]
# # (ret)

# plt.figure(figsize = (20, 4))

# # plt.subplot(1, 3, 1)
# plt.plot(data)

# for o in ret :
#     plt.plot(o)


# # plt.title("Original Signal")
# # plt.subplot(1, 3, 2)
# # plt.title("Approximation Coefficients")
# # plt.subplot(1, 3, 3)
# # plt.plot(cD)
# # plt.title("Detail Coefficients")
# # plt.tight_layout()
# plt.show();

#### • Test de classification - Proposition inputs 02

In [ ]:
### • Test de classification

# pca = PCA()
# sc = StandardScaler()

# X = pca.fit_transform(X)

# display(np.shape(X))

# X = (trains['C4'] - trains['C3']) # / [v.max() for v in trains['Cz']]
# X = [sk_p.minmax_scale(pywt.dwt(v, wavelet = 'db4')[0]) for v in X]
# y = trains['EventType']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

X_train_scaled = X_train
X_test_scaled  = X_test

# X_train_scaled = sc.fit_transform(X_train)
# X_test_scaled  = sc.transform(X_test)

# csp = CSP(nfilter = 2)
# K-plus proches voisins
knc = neighbors.KNeighborsClassifier()
# SVM (support vector machine)[, 'auto', kernel = 'rbf']
clf = svm.SVC(gamma = 'scale')
# RandomForest 
rfc = ensemble.RandomForestClassifier(n_jobs = -1)
#
lrg = LogisticRegression()
#ExtraTreesClassifier 
# Voting_clf = VotingClassifier(estimators = [('knn', clf1), ('svm', clf2), ('rf', clf3)], voting = 'hard')
# cv3        = model_selection.KFold(n_splits = 3, random_state = 42, shuffle = True), clf4

# Create a pipeline
# pip = Pipeline([('RFC', rfc), ('SVM', clf)])    # ('CSP', csp), 

# pip.fit(X_train, y_train)

# y_pred = pip.predict(X_test)

# y_train = np.array(y_train)

# print("Classification report:\n", classification_report(y_test, y_pred))
# print("Accuracy score:", accuracy_score(y_test, y_pred)), cv = 5, lrg

for reg in [rfc, knc, clf] :
    scores : dict = cross_validate(reg, X_train, y_train, scoring = ['accuracy'])
    
    reg.fit(X_train_scaled, y_train)
    
    r = scores['test_accuracy']
    
    print(f"● {reg} : Accuracy -> {r.mean():.1%} (±{r.std():.2}, max : {r.max():.1%})")
    print(f"\t-> score / test : {reg.score(X_test_scaled, y_test):.1%}\n")

In [ ]:
b, y = bands_coeff['Theta']

i = np.random.randint(np.shape(X)[0])
# X = [np.sign(s) for s in (trains['C4'] - trains['C3'])]
# y = {band: bandpass_filter(X[i], b, a) for band, (b, a) in bands_coeff.items()}
y = trains['C4'][i] - trains['C3'][i]

plt.figure()
# plt.style.use('')

plt.plot(normalized(bandpass_filter(y, b, y)), label = 'C4 - C3')
plt.plot(normalized(bandpass_filter(np.sign(y), b, y)), label = '[C4 - C3]', c = np.random.rand(1, 3)[0])
# plt.plot(pywt.dwt(y, wavelet = 'db4')[0], label = 'C4 - C3')
# plt.plot(pywt.dwt(np.sign(y), wavelet = 'db4')[0], label = '[C4 - C3]')
# plt.plot(np.zeros(512), ls = '--', c = np.random.rand(1, 3)[0])

# print(np.shape(pywt.dwt(y, wavelet = 'db8')))

# plt.plot(bandpass_filter(np.sign(trains['C3'][i]), b, a), label = 'C3')
# plt.plot(bandpass_filter(np.sign(trains['C4'][i]), b, a), label = 'C4')
# plt.plot(bandpass_filter(np.sign(trains['Cz'][i]), b, a), label = 'Cz')

# for (band, signal) in reversed(y.items()) :
#     plt.plot(pd.Series(signal), label = f'{band}', c = np.random.rand(1, 3)[0])

plt.title(f"{i}")
plt.legend(loc = 'upper right')

plt.grid()
plt.tight_layout()
plt.show();

In [ ]:
params = {
    ## K-plus proches voisins
    'knn__n_neighbors' : range(2),
    ## SVM
    'svm__C'      : [0.1, 1, 5],
    'svm__kernel' : ['linear', 'softmax', 'sigmoid', 'rbf'],
    ## RandomForest
    # 'rf__max_features'      : ['sqrt', 'log2', None],
    # 'rf__min_samples_split' : range(2, 32, 2),
    # , ('rf', clf3), ('rf', clf3)
    'estimators': [[('knn', knc), ('svm', svm)], [('knn', knc), ('svm', svm)]] 
    }

grid = model_selection.GridSearchCV(estimator = Voting_clf, param_grid = params, cv = 5) \
    .fit(X_train_scaled, y_train)

# parametres = {'max_features' : ['log2', 'sqrt', None], 'min_samples_split' : range(2, 32, 2)}

# vclf = model_selection.GridSearchCV(estimator = clf3, param_grid = parametres, cv = 3) \
#     .fit(X_train_scaled, y_train)

print(grid.best_estimator_)
print(grid.best_score_)
print('score train :', grid.score(X_train_scaled, y_train))
print('score test :', grid.score(X_test_scaled, y_test))

# print(vclf.best_estimator_, vclf.best_params_, vclf.best_score_)
# print('score train :', grid.score(X_train_scaled, y_train), vclf.score(X_train_scaled, y_train))
# print('score test  :', grid.score(X_test_scaled, y_test), vclf.score(X_test_scaled, y_test))

df_train_cpy, event_start = fancy_df(train_csv, label_csv['EventType'], hands_event, SCOPE)

fig, axes = plt.subplots(1, 1, figsize = (24, 5), sharey = True)
sig = .05

axes.plot(train_csv['C3'])

for p in event_start :
    axes.axvspan(p[0] - (SCOPE >> 1), p[0] + 1.5 * SCOPE, facecolor = 'orangered', alpha = .5)

plt.xlabel('Frequency (Hz)')
plt.ylabel('Score')

In [ ]:
boolInt    = 16
start  = event_start[boolInt][0]
entrant  = start + SCOPE
df   = df_train_cpy['C3_4'][start : entrant]
smooth = df.copy()
inc      = 5
alpha  = 1 / 3
dec    = int(inc / alpha)

plt.figure(figsize = (24, 5))
plot_window(train_csv, ['C3', 'C4', 'C3 + C4'], start, SCOPE)

# Lissage des hautes fréquences
for _ in range(inc) :
    smooth = simple_exponential_smoothing(smooth, alpha, 0)

smooth = pd.Series(index = range(start, entrant + inc - dec), data = smooth[dec :])

# plt.plot(raw - smooth, label = hands[event_start[pos][1]])
plt.plot(smooth, '--', label = hands_event[event_start[boolInt][1]])
plt.legend()
plt.show()

In [ ]:
"""
- Introduction :  
L’électroencéphalogramme (EEG) est une technique d’imagerie cérébrale utilisée pour étudier les activités du cerveau.  
En plaçant des capteurs sur le cuir chevelu, l’activité électrique du cerveau est enregistrée,  
   ce qui permet de comprendre les fonctionnements cérébraux et d’identifier certains schémas que l’on peut ensuite attribuer à des comportements précis.  
Un des schémas d'EEG qui a été beaucoup étudié est l’imagerie motrice (IM), ou le mouvement imaginaire de la main.  
Les IM créent des schémas bien définis qui peuvent être détectés.  
Le but de ce projet est de créer et d’entraîner un programme permettant de prédire si l’IM d’une personne correspond à un mouvement de la main droite ou de la main gauche.  
# **2. Étapes du projet**
- Prétraitement des Données :  
Les données EEG sont sujettes à des artefacts ou des erreurs de collecte dues à des mouvements parasites ou des interférences.  
Il est donc nécessaire d'appliquer un système de prétraitement des données pour réduire le bruit et extraire les bandes de fréquences pertinentes.

- Segmentation des données et extraction des caractéristiques :  
Les données EEG sont présentées comme un flux continu. Il est donc important, pour une meilleure analyse, de diviser les données en segments temporels correspondant à l’IM.  
Ensuite, identifier et extraire les caractéristiques pertinentes des signaux EEG associées aux IM est essentiel.  
Cela comprend la puissance et d'autres spécificités de l’activité électrique qui définissent les IM.

- Analyse statistique exploratoire :  
Utiliser les outils d’analyse exploratoire pour mieux comprendre les données et identifier les tendances ou les patterns significatifs.

- Entraînement du modèle :  
Entraîner un modèle permettant de distinguer les différences entre les IM des mains droite et gauche.  
Optimiser le modèle et évaluer sa performance sur un ensemble de test.

- Conclusion :  
Ces étapes sont cruciales pour développer un programme efficace de prédiction des mouvements imaginaires de la main basé sur les données EEG.
"""

"""
parts = []        #
temp = [[], []]  # Les époques pour tous les cannaux et tous les évènements.
spots = [[], []]  # Apparitions des évènements

# Pour la standardisation du nombre d'échantillon max conservé
ceil   = min([len(label_csv[i]['EventType']) for i in count])
merge  = False

# Extraction des données relavitives à l'apparition des évènements.
for i in count :
    df   = train_csv[i]
    kind  = label_csv[i]['EventType'][: ceil]
    loc = np.where(df['EventStart'] == 1)[: ceil]

    parts.append(zero_removal(df['C3'], 75))

    for i in num_events :
        spots[i].append(np.array(*loc)[*np.where(kind == i)])

        room = event_epochs(spots[i][-1], SCOPE, LAG)

        temp[i].append([full_event(df[c], room, merge)for c in eeg_Channels])

# Regroupement des données en fonction du type de l'évènement et du cannal d'observation
if merge :
    temp = [[[np.append([], T[j :: 3]) for T in temp[i]] for j in range(3)] for i in num_events]
else :
    store = [[[], [], []], [[], [], []]]
    
    [[[[store[i][j].append(G) for G in X] for j, X in enumerate(T)] for T in temp[i]] for i in num_events]

    temp = store
    # res = [[np.concatenate(np.stack(tries[i], axis = 1)[j], axis = 0) for j in range(3)] for i in n_type]

eras    = [pd.DataFrame(dict(zip(eeg_Channels, [pd.Series(X) for X in temp[i]]))) for i in num_events]
"""

"""
# display(df_left)
# display(df_right)

# left = np.array([[f(v) for v in tmp[c]] for c in eeg_Chans for f in foo])
# display(pd.DataFrame(left.T, columns = [f"L{c}_{x.__name__}" for x in foo for c in eeg_Chans])), columns = [f"L{c}_{x.__name__}" for x in foo for c in eeg_Chans]

# print(np.shape(left_df))

# right = np.array([[f(v) for v in tmp[c]] for c in eeg_Chans for f in foo])
# display(pd.DataFrame(right.T, columns = [f"R{c}_{x.__name__}" for x in foo for c in eeg_Chans]))

# left[1] - left[0], left[4] - left[3], left[7] - left[6], left[10] - left[9], left[13] - left[12]
# left[0], left[1]

# left_min = [apply_func(tmp[c], np.min) for c in eeg_Chans]apply_func(tmp[c], f)
# left_max = [apply_func(tmp[c], np.max) for c in eeg_Chans]
# left_std = [apply_func(tmp[c], np.std) for c in eeg_Chans]
# left_ptp = [apply_func(tmp[c], np.ptp) for c in eeg_Chans]
# left_men = [apply_func(tmp[c], np.mean) for c in eeg_Chans]
# left_med = [apply_func(tmp[c], np.median) for c in eeg_Chans]

# right_min = [apply_func(tmp[c], np.min) for c in eeg_Chans]
# right_max = [apply_func(tmp[c], np.max) for c in eeg_Chans]
# right_std = [apply_func(tmp[c], np.std) for c in eeg_Chans]
# right_ptp = [apply_func(tmp[c], np.ptp) for c in eeg_Chans]
# right_men = [apply_func(tmp[c], np.mean) for c in eeg_Chans]
# right_med = [apply_func(tmp[c], np.median) for c in eeg_Chans]

# display(pd.DataFrame(np.array((*left_min, *left_max, *left_std, *left_ptp, *left_men, *left_med)).T))
# display(pd.DataFrame(np.array((*right_min, *right_max, *right_std, *right_ptp, *right_men, *right_med)).T))

# pd.DataFrame(np.array((lc3_i, rc3_i, lc3_a, rc3_a, lc3_p, rc3_p, lc3_m, rc3_m,
#                        lc4_i, rc4_i, lc4_a, rc4_a, lc4_p, rc4_p, lc4_m, rc4_m,
#                        lcz_i, rcz_i, lcz_a, rcz_a, lcz_p, rcz_p, lcz_m, rcz_m)).T)

# lc3_i = np.array(apply_func(left['C3'], np.min))
# lc4_i = np.array(apply_func(left['C4'], np.min))
# lcz_i = np.array(apply_func(left['Cz'], np.min))

# lc3_a = np.array(apply_func(left['C3'], np.max))
# lc4_a = np.array(apply_func(left['C4'], np.max))
# lcz_a = np.array(apply_func(left['Cz'], np.max))

# lc3_m = np.array(apply_func(left['C3'], np.mean))
# lc4_m = np.array(apply_func(left['C4'], np.mean))
# lcz_m = np.array(apply_func(left['Cz'], np.mean))

# rc3_i = np.array(apply_func(right['C3'], np.min))
# rc4_i = np.array(apply_func(right['C4'], np.min))
# rcz_i = np.array(apply_func(right['Cz'], np.min))

# rc3_a = np.array(apply_func(right['C3'], np.max))
# rc4_a = np.array(apply_func(right['C4'], np.max))
# rcz_a = np.array(apply_func(right['Cz'], np.max))

# rc3_m = np.array(apply_func(right['C3'], np.mean))
# rc4_m = np.array(apply_func(right['C4'], np.mean))
# rcz_m = np.array(apply_func(right['Cz'], np.mean))

# plt.scatter(x = lc3_i, y = lc3_a)
# plt.scatter(x = rc3_i, y = rc3_a)
# plt.scatter(x = lc4_i, y = lc4_a)
# plt.scatter(x = rc4_i, y = rc4_a)
"""

# def apply_func(data : Board, foo) -> Vector : return [foo(v) for v in data]